In [1]:
suppressMessages(library(ArchR))
suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(harmony))
suppressMessages(library(dplyr))
suppressMessages(library(cowplot))
suppressMessages(library(harmony))
suppressMessages(library(Nebulosa))
suppressMessages(library(ggpubr))
suppressMessages(library(Ipaper))
suppressMessages(library(phateR))
suppressMessages(library(parallel))

In [2]:
set.seed(42)
getwd()

addArchRThreads(threads = parallel::detectCores() - 2)
addArchRGenome("hg38")

[1] "/data/scATA/SingleCellOpenChromatin/local/VisiumHeartRevision/IntegrativeAnalysis/Fibroblast/code"

Setting default number of Parallel threads to 126.

Setting default genome to Hg38.



In [12]:
df.matching <- readRDS("../data/coembed/ATAC_RNA_matching.rds")
head(df.matching)

,ATAC,RNA,cell_name
,<chr>,<chr>,<chr>
1,CK171#CGGACCAGTGAGTTGG-1,GTAGCTACAAGCGCTC-1_2_1_1_1_1,cell-1
2,CK171#AAAGGGCAGGTAGGAA-1,GGGACTCGTATCTCTT-1_1_1_1_1_1_1_1_1_1_1_1_1,cell-2
3,CK171#GTAGTACAGGCCTAAG-1,TTATTGCTCATAGGCT-1_1_1_1,cell-3
4,CK171#GGGTCTGTCGGGTCCA-1,CCGTTCACACCTGAAT-1_2_1_1_1_1_1_1,cell-4
5,CK171#TTCGATTGTGCAACAG-1,TGGTACAGTCAGATTC-1_1_1,cell-5
6,CK171#GTAGACTCATAGCCAT-1,GCCAGTGGTAAGCAAT-1_1_1_1_1_1_1_1_1_1_1_1_1_1,cell-6


In [4]:
coembed <- readRDS("../data/coembed/coembed.annotation.trajectory.Rds")
obj.atac <- coembed[, df.matching$ATAC]

In [8]:
obj.atac

An object of class Seurat 
221802 features across 4823 samples within 3 assays 
Active assay: RNA (28933 features, 2000 variable features)
 2 other assays present: peaks, GeneActivity
 7 dimensional reductions calculated: pca, umap, harmony, umap_harmony, umap_harmony_v2, phate, dm

In [9]:
obj.atac <- obj.atac[, !is.na(obj.atac$SCARA5_Myofib_dm)]

In [10]:
obj.atac

An object of class Seurat 
221802 features across 2082 samples within 3 assays 
Active assay: RNA (28933 features, 2000 variable features)
 2 other assays present: peaks, GeneActivity
 7 dimensional reductions calculated: pca, umap, harmony, umap_harmony, umap_harmony_v2, phate, dm

In [ ]:
## subset atac project, here we only use the cells that 
## are in the trajectory from SCARA5+ fibroblasts to myofibroblast
proj <- loadArchRProject("../data/snATAC", showLogo = FALSE)

proj <- subsetArchRProject(proj, 
                           cells = colnames(obj.atac),
                           outputDirectory = "../data/snATAC_SCARA5_Myofib",
                           force = TRUE)

In [ ]:
## add cell states information
meta.data <- as.data.frame(obj.atac@meta.data)
meta.data <- meta.data[proj@cellColData@rownames, ]
annotation <- meta.data$annotation

proj <- addCellColData(proj, 
                       data = as.character(annotation), 
                        cells = rownames(meta.data),
                       name = "annotation", 
                       force = TRUE)

In [15]:
df.matching.sub <- df.matching %>%
    dplyr::filter(ATAC %in% colnames(obj.atac))

obj.rna <- subset(coembed[, df.matching.sub$RNA])

obj.rna

An object of class Seurat 
221802 features across 2082 samples within 3 assays 
Active assay: RNA (28933 features, 2000 variable features)
 2 other assays present: peaks, GeneActivity
 7 dimensional reductions calculated: pca, umap, harmony, umap_harmony, umap_harmony_v2, phate, dm

In [ ]:
## add gene expression
geneMatrix <- getMatrixFromProject(ArchRProj = proj,
                                   useMatrix = "GeneScoreMatrix")

gex.mat <- as.matrix(obj.rna@assays$RNA@counts)
colnames(gex.mat) <- df.matching.sub$ATAC

rowRanges <- rowData(geneMatrix)
sel_genes <- intersect(rownames(gex.mat), rowRanges$name)

length(sel_genes)

gex.mat <- gex.mat[sel_genes, ]
rownames(rowRanges) <- rowRanges$name
rowRanges <- rowRanges[sel_genes, ]

rowRanges <- GRanges(rowRanges$seqnames,
                     IRanges(start = as.numeric(rowRanges$start),
                             end = as.numeric(rowRanges$start) + 1))

seRNA <- SummarizedExperiment(assays = SimpleList(counts = gex.mat),
                              rowRanges = rowRanges)

proj <- addGeneExpressionMatrix(proj,
                                seRNA = seRNA,
                                force = TRUE)

In [ ]:
saveRDS(obj.atac, file = "../data/snATAC_SCARA5_Myofib/snATAC.Rds")

saveArchRProject(ArchRProj = proj, 
                 load = FALSE)

In [ ]:
sessionInfo()